In [3]:
from typing import List
def get_requirements()-> List:
    """

    This function returns a list of requirements in list format
    """
    try:
        requirement_list:List[str] = []

        with open('requirements.txt', 'r') as f:
            lines = f.readlines()
        requirement_list = [line.replace('\n', '') for line in lines]
        if "-e ." in requirement_list:
            requirement_list.remove("-e .")

        return requirement_list
    except Exception as e:
        raise e

In [4]:
get_requirements()

['pymongo[srv]==4.2.0']